# 4️⃣ Benchmark: 표준 벤치마크 평가

이 노트북은 다음을 수행합니다:
1. PTH 모델 벤치마크 평가 (Set5, Set14, B100, Urban100, Manga109)
2. PTL 모델 벤치마크 평가
3. 결과 저장 및 비교 시각화

**평가 대상:**
- 원본 CATANet-L x2 (PTH + PTL)
- Output KD (PTH + PTL)
- Feature KD (PTH + PTL)
- FaKD (PTH + PTL)

**논문 Table 2 (CATANet-L ×2) 기준값:**
- Set5: 38.28 dB
- Set14: 33.99 dB
- B100: 32.37 dB
- Urban100: 33.09 dB
- Manga109: 39.37 dB

## 4.0 환경 변수 설정

In [ ]:
import os
import sys

# 상대 경로 설정
project_dir = os.path.dirname(os.path.abspath('__file__'))
if not project_dir or project_dir == '.':
    project_dir = os.getcwd()

# 가상환경 경로 설정
home_dir = os.path.expanduser('~')
venv_path = os.path.join(home_dir, 'my_ssl/ssl_env')
site_packages = os.path.join(venv_path, 'lib', 'python3.12', 'site-packages')
python_exec = os.path.join(venv_path, 'bin', 'python')

if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

%cd {project_dir}

# 결과 저장 디렉토리 생성
os.makedirs('results/benchmark', exist_ok=True)

print(f"✅ 환경 설정 완료: {project_dir}")

## 4.1 평가 대상 모델 확인

In [ ]:
print("="*70)
print("📁 평가 대상 모델 확인")
print("="*70)

# PTH 모델
pth_models = [
    ('Original', 'weights/CATANet-L_x2.pth'),
    ('Output_KD', 'weights/finetuned_output.pth'),
    ('Feature_KD', 'weights/finetuned_feature.pth'),
    ('FaKD', 'weights/finetuned_fakd.pth'),
]

print("\n🔹 PTH 모델:")
for name, path in pth_models:
    if os.path.exists(path):
        size = os.path.getsize(path) / (1024 * 1024)
        print(f"   ✅ {name:12} | {path} ({size:.1f} MB)")
    else:
        print(f"   ❌ {name:12} | {path} (없음)")

# PTL 모델
print("\n🔹 PTL 모델:")
ptl_count = 0
if os.path.exists('weights'):
    ptl_files = [f for f in os.listdir('weights') if f.endswith('.ptl')]
    for f in sorted(ptl_files):
        size = os.path.getsize(f'weights/{f}') / (1024 * 1024)
        print(f"   ✅ {f} ({size:.1f} MB)")
        ptl_count += 1
    if ptl_count == 0:
        print("   ❌ PTL 파일 없음")

print(f"\n📊 총 PTL 파일: {ptl_count}개")

## 4.2 벤치마크 데이터셋 확인

In [ ]:
benchmark_dir = os.path.join(project_dir, 'datasets/benchmark')

print("📁 벤치마크 데이터셋 확인:")
if os.path.exists(benchmark_dir):
    !ls -la {benchmark_dir}
else:
    print("⚠️ 벤치마크 데이터셋이 없습니다!")
    print("   1_setup.ipynb를 먼저 실행하세요.")

---
## 4.3 PTH 모델 벤치마크 평가

In [ ]:
print("="*70)
print("🎯 PTH 모델 벤치마크 평가")
print("="*70)
print("\n📊 논문 기준값 (CATANet-L ×2):")
print("   Set5: 38.28 | Set14: 33.99 | B100: 32.37 | Urban100: 33.09 | Manga109: 39.37")
print("="*70)

# 평가할 PTH 모델 목록
pth_models = [
    ('Original', 'weights/CATANet-L_x2.pth'),
    ('Output_KD', 'weights/finetuned_output.pth'),
    ('Feature_KD', 'weights/finetuned_feature.pth'),
    ('FaKD', 'weights/finetuned_fakd.pth'),
]

output_csv = 'results/benchmark/pth_benchmark_results.csv'

for name, path in pth_models:
    if os.path.exists(path):
        print(f"\n{'#'*60}")
        print(f"# {name}: {path}")
        print(f"{'#'*60}")
        
        !{python_exec} evaluate_benchmark.py \
            --weights {path} \
            --benchmark_dir datasets/benchmark \
            --scale 2 \
            --model_name "{name}_PTH" \
            --output {output_csv}
    else:
        print(f"\n⚠️ {name} 없음: {path}")

print("\n" + "="*70)
print(f"✅ PTH 벤치마크 결과 저장: {output_csv}")

---
## 4.4 PTL 모델 벤치마크 평가

PTL 파일은 trace된 고정 입력 크기에서만 동작하므로, 벤치마크 평가 시 이미지를 해당 크기로 리사이즈하여 테스트합니다.

In [ ]:
# PTL 벤치마크 평가 스크립트 생성
ptl_benchmark_script = '''
#!/usr/bin/env python3
import os
import sys
import argparse
import time
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import csv
from datetime import datetime


def rgb2ycbcr(img):
    if isinstance(img, torch.Tensor):
        img = img.cpu().numpy()
    if img.ndim == 4:
        img = img[0]
    if img.shape[0] == 3:
        img = np.transpose(img, (1, 2, 0))
    img = img.astype(np.float64)
    y = 16. + (65.481 * img[:,:,0] + 128.553 * img[:,:,1] + 24.966 * img[:,:,2]) / 255.
    return y


def calculate_psnr(img1, img2, border=0):
    if border > 0:
        img1 = img1[border:-border, border:-border]
        img2 = img2[border:-border, border:-border]
    mse = np.mean((img1.astype(np.float64) - img2.astype(np.float64)) ** 2)
    if mse == 0:
        return float("inf")
    return 20 * np.log10(255.0 / np.sqrt(mse))


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--ptl_path", type=str, required=True)
    parser.add_argument("--input_size", type=int, required=True)
    parser.add_argument("--benchmark_dir", type=str, default="datasets/benchmark")
    parser.add_argument("--scale", type=int, default=2)
    parser.add_argument("--output", type=str, default="results/benchmark/ptl_benchmark_results.csv")
    parser.add_argument("--model_name", type=str, default="PTL_Model")
    args = parser.parse_args()
    
    device = torch.device("cpu")
    print(f"Device: {device}")
    print(f"Input size: {args.input_size}x{args.input_size}")
    
    print(f"Loading: {args.ptl_path}")
    model = torch.jit.load(args.ptl_path, map_location=device)
    model.eval()
    
    datasets = ["Set5", "Set14"]
    results = {}
    inference_times = []
    
    print(f"\\n{"="*50}")
    print(f"Benchmark Evaluation (Scale: x{args.scale})")
    print(f"{"="*50}")
    
    for ds in datasets:
        ds_path = os.path.join(args.benchmark_dir, ds)
        hr_dir = os.path.join(ds_path, "HR")
        lr_dir = os.path.join(ds_path, f"LR_bicubic", f"X{args.scale}")
        
        if not os.path.exists(hr_dir) or not os.path.exists(lr_dir):
            continue
        
        hr_images = sorted([f for f in os.listdir(hr_dir) if f.endswith((".png", ".jpg", ".bmp"))])
        psnr_list = []
        
        for hr_name in tqdm(hr_images, desc=ds, leave=False):
            base_name = os.path.splitext(hr_name)[0]
            lr_path = os.path.join(lr_dir, f"{base_name}x{args.scale}.png")
            hr_path = os.path.join(hr_dir, hr_name)
            
            if not os.path.exists(lr_path):
                continue
            
            lr_pil = Image.open(lr_path).convert("RGB")
            lr_resized = lr_pil.resize((args.input_size, args.input_size), Image.BICUBIC)
            lr_np = np.array(lr_resized).astype(np.float32) / 255.0
            lr_tensor = torch.from_numpy(np.transpose(lr_np, (2, 0, 1))).unsqueeze(0)
            
            start_time = time.time()
            with torch.no_grad():
                sr_tensor = model(lr_tensor).clamp(0, 1)
            inference_times.append(time.time() - start_time)
            
            hr_pil = Image.open(hr_path).convert("RGB")
            hr_resized = hr_pil.resize((args.input_size * args.scale, args.input_size * args.scale), Image.BICUBIC)
            hr_np = np.array(hr_resized).astype(np.float32) / 255.0
            
            sr_y = rgb2ycbcr(sr_tensor.numpy()[0] * 255)
            hr_y = rgb2ycbcr(np.transpose(hr_np, (2, 0, 1)) * 255)
            
            psnr_list.append(calculate_psnr(sr_y, hr_y, border=args.scale))
        
        if psnr_list:
            results[ds] = np.mean(psnr_list)
            print(f"{ds:12} PSNR: {results[ds]:.4f} dB")
    
    avg_time = np.mean(inference_times) * 1000 if inference_times else 0
    print(f"\\nAvg inference time: {avg_time:.2f} ms")
    
    os.makedirs(os.path.dirname(args.output), exist_ok=True)
    file_exists = os.path.exists(args.output)
    with open(args.output, "a", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            header = ["Timestamp", "Model", "Input_Size", "Set5_PSNR", "Set14_PSNR", "Avg_Time_ms"]
            writer.writerow(header)
        
        row = [
            datetime.now().strftime("%Y-%m-%d %H:%M"),
            args.model_name,
            args.input_size,
            f"{results.get("Set5", 0):.4f}",
            f"{results.get("Set14", 0):.4f}",
            f"{avg_time:.2f}"
        ]
        writer.writerow(row)
    
    print(f"\\n✅ Results saved to {args.output}")


if __name__ == "__main__":
    main()
'''

script_path = os.path.join(project_dir, 'evaluate_ptl_benchmark.py')
with open(script_path, 'w') as f:
    content = ptl_benchmark_script.replace('\\n', '\n').replace('\\"', '"')
    f.write(content)

print(f"✅ PTL 벤치마크 스크립트 생성: {script_path}")

In [ ]:
print("="*70)
print("🎯 PTL 모델 벤치마크 평가")
print("="*70)

# PTL 파일 목록
ptl_models = []
if os.path.exists('weights'):
    for f in os.listdir('weights'):
        if f.endswith('.ptl'):
            parts = f.replace('.ptl', '').split('_')
            try:
                input_size = int(parts[-1])
                model_name = '_'.join(parts[:-1])
                ptl_models.append((model_name, f'weights/{f}', input_size))
            except:
                pass

output_csv = 'results/benchmark/ptl_benchmark_results.csv'

if len(ptl_models) == 0:
    print("⚠️ PTL 파일이 없습니다!")
    print("   1_setup.ipynb와 3_ptl.ipynb를 먼저 실행하세요.")
else:
    ptl_models.sort(key=lambda x: (x[0], x[2]))
    
    for model_name, ptl_path, input_size in ptl_models:
        print(f"\n{'#'*60}")
        print(f"# {model_name} (입력: {input_size}x{input_size})")
        print(f"{'#'*60}")
        
        !{python_exec} evaluate_ptl_benchmark.py \
            --ptl_path {ptl_path} \
            --input_size {input_size} \
            --benchmark_dir datasets/benchmark \
            --scale 2 \
            --model_name "{model_name}" \
            --output {output_csv}

print("\n" + "="*70)
print(f"✅ PTL 벤치마크 결과 저장: {output_csv}")

---
## 4.5 결과 비교 테이블

In [ ]:
import pandas as pd
from IPython.display import display

print("="*80)
print("📊 벤치마크 결과 비교")
print("="*80)

# PTH 결과
pth_results = 'results/benchmark/pth_benchmark_results.csv'
if os.path.exists(pth_results):
    print("\n🔹 PTH 모델 결과:")
    df_pth = pd.read_csv(pth_results)
    display(df_pth)
else:
    print("\n⚠️ PTH 벤치마크 결과 없음")

# PTL 결과
ptl_results = 'results/benchmark/ptl_benchmark_results.csv'
if os.path.exists(ptl_results):
    print("\n🔹 PTL 모델 결과:")
    df_ptl = pd.read_csv(ptl_results)
    display(df_ptl)
else:
    print("\n⚠️ PTL 벤치마크 결과 없음")

# 논문 기준값
print("\n🔹 논문 기준값 (CATANet-L ×2):")
print("   Set5: 38.28 | Set14: 33.99 | B100: 32.37 | Urban100: 33.09 | Manga109: 39.37")

---
## 4.6 결과 시각화

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("📊 벤치마크 결과 시각화")
print("="*60)

# PTH 결과 시각화
pth_results = 'results/benchmark/pth_benchmark_results.csv'
if os.path.exists(pth_results):
    df = pd.read_csv(pth_results)
    
    paper = {'Set5': 38.28, 'Set14': 33.99, 'B100': 32.37, 'Urban100': 33.09, 'Manga109': 39.37}
    datasets = ['Set5', 'Set14', 'B100', 'Urban100', 'Manga109']
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    x = np.arange(len(datasets))
    width = 0.15
    
    paper_vals = [paper[ds] for ds in datasets]
    ax.bar(x - width*2, paper_vals, width, label='Paper', color='gray', alpha=0.7)
    
    colors = ['#3B82F6', '#10B981', '#F59E0B', '#EF4444']
    for idx, (_, row) in enumerate(df.iterrows()):
        if idx >= len(colors):
            break
        vals = []
        for ds in datasets:
            col = f'{ds}_PSNR'
            try:
                vals.append(float(row[col]) if col in row and pd.notna(row[col]) and row[col] != 'N/A' else 0)
            except:
                vals.append(0)
        ax.bar(x + width*(idx-1), vals, width, label=row['Model'], color=colors[idx])
    
    ax.set_ylabel('PSNR (dB)', fontsize=12)
    ax.set_xlabel('Dataset', fontsize=12)
    ax.set_title('PTH Model Benchmark Comparison', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(datasets)
    ax.legend(loc='lower right')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(30, 42)
    
    plt.tight_layout()
    plt.savefig('results/benchmark/pth_comparison.png', dpi=150)
    plt.show()
    
    print("\n✅ 그래프 저장: results/benchmark/pth_comparison.png")
else:
    print("⚠️ PTH 결과 파일 없음")

# PTL 추론 시간 비교
ptl_results = 'results/benchmark/ptl_benchmark_results.csv'
if os.path.exists(ptl_results):
    df_ptl = pd.read_csv(ptl_results)
    
    if len(df_ptl) > 0:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        models = df_ptl['Model'].unique()
        colors = plt.cm.Set2(np.linspace(0, 1, len(models)))
        
        for idx, model in enumerate(models):
            model_data = df_ptl[df_ptl['Model'] == model].sort_values('Input_Size')
            ax.plot(model_data['Input_Size'], model_data['Avg_Time_ms'], 
                    marker='o', label=model, color=colors[idx], linewidth=2)
        
        ax.set_xlabel('Input Size', fontsize=12)
        ax.set_ylabel('Inference Time (ms)', fontsize=12)
        ax.set_title('PTL Model Inference Time Comparison', fontsize=14)
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('results/benchmark/ptl_inference_time.png', dpi=150)
        plt.show()
        
        print("\n✅ 그래프 저장: results/benchmark/ptl_inference_time.png")

---
## 4.7 최종 요약

In [ ]:
import pandas as pd

print("="*80)
print("📊 최종 벤치마크 요약")
print("="*80)

# 결과 파일 확인
print("\n📁 생성된 결과 파일:")
!ls -lh results/benchmark/ 2>/dev/null || echo "   결과 파일 없음"

# PTH 요약
pth_results = 'results/benchmark/pth_benchmark_results.csv'
if os.path.exists(pth_results):
    df = pd.read_csv(pth_results)
    print("\n🔹 PTH 모델 Set5 PSNR:")
    for _, row in df.iterrows():
        if 'Set5_PSNR' in row and pd.notna(row['Set5_PSNR']):
            try:
                psnr = float(row['Set5_PSNR'])
                print(f"   {row['Model']:20} | {psnr:.4f} dB")
            except:
                pass

# PTL 요약
ptl_results = 'results/benchmark/ptl_benchmark_results.csv'
if os.path.exists(ptl_results):
    df = pd.read_csv(ptl_results)
    print("\n🔹 PTL 모델 추론 시간 (640x640 기준):")
    df_640 = df[df['Input_Size'] == 640]
    for _, row in df_640.iterrows():
        print(f"   {row['Model']:20} | {row['Avg_Time_ms']:.2f} ms")

print("\n" + "="*80)
print("✅ 벤치마크 평가 완료!")
print("="*80)